In [1]:
# ----------------------
# Απαιτούμενα πακέτα (σε Colab: !pip install yfinance ipywidgets matplotlib numpy pandas scipy)
# ----------------------

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.optimize import minimize
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display

# ----------------------
# Widgets για είσοδο
# ----------------------
months_slider = widgets.IntSlider(
    value=12,
    min=1,
    max=600,
    step=1,
    description='Μήνες:',
    continuous_update=False
)

capital_input = widgets.FloatText(
    value=100000,
    description='Κεφάλαιο (€):',
    disabled=False
)

confidence_slider = widgets.FloatSlider(
    value=0.95,
    min=0.8,
    max=0.99,
    step=0.01,
    description='Επίπεδο εμπιστοσύνης:',
    readout_format='.2f'
)

stock_input = widgets.Text(
    value='',
    placeholder='Εισάγετε μετοχές (π.χ. AAPL,MSFT,TSLA)',
    description='Μετοχές:',
    disabled=False
)

button = widgets.Button(description="Υπολογισμός")
output = widgets.Output()

# ----------------------
# Συνάρτηση υπολογισμού
# ----------------------
def calculate(button):
    output.clear_output()

    months_back = months_slider.value
    ticker_input = stock_input.value.strip()
    capital = capital_input.value
    confidence_level = confidence_slider.value

    if not ticker_input:
        with output:
            print("❗Παρακαλώ εισάγετε τουλάχιστον μία μετοχή.")
        return

    tickers_input_order = [t.strip().upper() for t in ticker_input.split(',')]

    end_date = datetime.today() - timedelta(days=1)
    start_date = end_date - timedelta(days=months_back * 30)

    with output:
        print(f"Λήψη δεδομένων από {start_date.strftime('%Y-%m-%d')} μέχρι {end_date.strftime('%Y-%m-%d')}")
        print("Tickers (input order):", tickers_input_order)

        try:
            data = yf.download(tickers_input_order, start=start_date, end=end_date)['Close']

            if data.empty:
                print("❗Δεν βρέθηκαν δεδομένα.")
                return

            # Διασφάλιση της σειράς των tickers
            valid_tickers = [col for col in tickers_input_order if col in data.columns]
            if not valid_tickers:
                print("❗Δεν βρέθηκαν δεδομένα για κανένα ticker.")
                return

            data = data[valid_tickers]  # Ταξινόμηση με βάση τη σειρά εισόδου
            returns = data.pct_change().dropna()

            annual_returns = returns.mean() * 252
            annual_cov_matrix = returns.cov() * 252
            num_assets = len(valid_tickers)

            # Βοηθητικές συναρτήσεις
            def portfolio_stats(weights, mean_returns, cov_matrix):
                portfolio_return = np.sum(mean_returns * weights)
                portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
                return portfolio_return, portfolio_volatility

            def minimize_volatility(weights, mean_returns, cov_matrix):
                return portfolio_stats(weights, mean_returns, cov_matrix)[1]

            def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0):
                ret, vol = portfolio_stats(weights, mean_returns, cov_matrix)
                sharpe = (ret - risk_free_rate) / vol
                return -sharpe

            constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            bounds = tuple((0, 1) for _ in range(num_assets))
            initial_guess = [1. / num_assets] * num_assets

            # Minimum Variance Portfolio
            res_min_var = minimize(minimize_volatility, initial_guess,
                                    args=(annual_returns, annual_cov_matrix),
                                    method='SLSQP', bounds=bounds, constraints=constraints)
            min_var_weights = res_min_var.x
            min_ret, min_vol = portfolio_stats(min_var_weights, annual_returns, annual_cov_matrix)

            # Optimal Portfolio
            res_opt = minimize(neg_sharpe_ratio, initial_guess,
                                args=(annual_returns, annual_cov_matrix, 0),
                                method='SLSQP', bounds=bounds, constraints=constraints)
            opt_weights = res_opt.x
            opt_ret, opt_vol = portfolio_stats(opt_weights, annual_returns, annual_cov_matrix)

            # Value at Risk (VaR)
            z_score = norm.ppf(1 - confidence_level)
            var_opt_daily = -(opt_ret / 252 + z_score * (opt_vol / np.sqrt(252)))
            var_min_daily = -(min_ret / 252 + z_score * (min_vol / np.sqrt(252)))

            var_opt_eur = capital * var_opt_daily
            var_min_eur = capital * var_min_daily

            # Γράφημα Efficient Frontier
            vol_range = np.linspace(min_vol, max(annual_returns), 100)
            sharpe_line = [opt_ret / opt_vol * v for v in vol_range]
            plt.figure(figsize=(10, 6))
            plt.plot(vol_range, sharpe_line, color='red', linestyle='--', label='Capital Market Line')
            plt.scatter(min_vol, min_ret, color='blue', marker='o', s=100, label='Min Variance Portfolio')
            plt.scatter(opt_vol, opt_ret, color='red', marker='*', s=200, label='Optimal Portfolio')
            plt.xlabel('Annual Risk (Volatility)')
            plt.ylabel('Annual Return')
            plt.title('Efficient Frontier')
            plt.legend()
            plt.grid(True)
            plt.show()

            # Εκτύπωση αποτελεσμάτων - Optimal
            print("\nΒέλτιστο Πορτοφόλιο (με τον υψηλότερο Sharpe Ratio):")
            for ticker, weight in zip(valid_tickers, opt_weights):
                print(f"{ticker}: {weight:.4f} ({weight*100:.2f}%)")
            print(f"\nΑναμενόμενη Ετήσια Απόδοση: {opt_ret:.4f} ({opt_ret*100:.2f}%)")
            print(f"Αναμενόμενη Ετήσια Volatility: {opt_vol:.4f} ({opt_vol*100:.2f}%)\n")
            print(f"Ημερήσιο Value at Risk ({confidence_level*100:.0f}%): {var_opt_daily:.4f} ({var_opt_daily*100:.2f}%)")
            print(f"Ημερήσιο VaR σε € (για κεφάλαιο €{capital}): €{-var_opt_eur:.2f}")

            # Εκτύπωση αποτελεσμάτων - Min Variance
            print("\n\nΧαρτοφυλάκιο Ελάχιστης Διακύμανσης:")
            for ticker, weight in zip(valid_tickers, min_var_weights):
                print(f"{ticker}: {weight:.4f} ({weight*100:.2f}%)")
            print(f"\nΑναμενόμενη Ετήσια Απόδοση: {min_ret:.4f} ({min_ret*100:.2f}%)")
            print(f"Αναμενόμενη Ετήσια Volatility: {min_vol:.4f} ({min_vol*100:.2f}%)\n")
            print(f"Ημερήσιο Value at Risk ({confidence_level*100:.0f}%): {var_min_daily:.4f} ({var_min_daily*100:.2f}%)")
            print(f"Ημερήσιο VaR σε € (για κεφάλαιο €{capital}): €{-var_min_eur:.2f}")

        except Exception as e:
            print("Σφάλμα:", str(e))

# ----------------------
# Σύνδεση κουμπιού
# ----------------------
button.on_click(calculate)

# ----------------------
# Εμφάνιση interface
# ----------------------
display(
    months_slider,
    capital_input,
    confidence_slider,
    stock_input,
    button,
    output
)

IntSlider(value=12, continuous_update=False, description='Μήνες:', max=600, min=1)

FloatText(value=100000.0, description='Κεφάλαιο (€):')

FloatSlider(value=0.95, description='Επίπεδο εμπιστοσύνης:', max=0.99, min=0.8, step=0.01)

Text(value='', description='Μετοχές:', placeholder='Εισάγετε μετοχές (π.χ. AAPL,MSFT,TSLA)')

Button(description='Υπολογισμός', style=ButtonStyle())

Output()